In [1]:
!pip install jellyfish
!pip install faker

In [2]:
import pyspark
import jellyfish
import pandas as pd
import numpy as np
from typing import List
import os
import math
from itertools import combinations, product
import time
from concurrent.futures import ProcessPoolExecutor
from collections import defaultdict
from packages.generateDataSets import SyntheticMatcherDataset
from packages.calculateStatistics import DatasetEvaluator

In [ ]:
dataset = SyntheticMatcherDataset(size=2500, true_positive_ratio=0.70)
df1, df2 = dataset.df1, dataset.df2
expected = dataset.expected

evaluator = DatasetEvaluator(df1, df2, expected, threshold=3, match_column="id")
evaluator.evaluate()
evaluator.printResults()

In [ ]:

# Data for df1
data1 = [
    ['ID00005', 'N039', 'E298', 'Q412', 'V409', 'R232'], #TP1
    ['ID00009', 'R822', 'W179', 'H017', 'P323', 'F298'], #TP2
    ['ID00007', 'R449', 'X716', 'M948', 'G667', 'S702'], #TP3
    ['ID00004', 'Q071', 'E396', 'N843', 'I458', 'S719'], #TP4
    ['ID10004', 'Q072', 'E496', 'N853', 'I623', 'S569'], #FN1
    ['NEW72378', 'J547', 'B222', 'G492', 'R551', 'S490'], #FP1
    ['ID00008', 'N322', 'K685', 'T442', 'C825', 'W967'], #FP2
    ['ID00000', 'W815', 'L281', 'R155', 'F768', 'B914'],
    ['ID00001', 'C172', 'B326', 'X400', 'M508', 'O776'],
    ['ID00002', 'V683', 'C265', 'J127', 'D589', 'F482'],
    ['ID00003', 'E851', 'P721', 'F745', 'D863', 'K229'],
    ['ID00016', 'T873', 'D670', 'U046', 'Z181', 'X621'],
    ['ID00017', 'F327', 'G856', 'E567', 'O929', 'Q721'],
    ['ID00010', 'O283', 'T723', 'Z034', 'V319', 'X338'],
]

# Data for df2
data2 = [
    ['ID00005', 'R746', 'E298', 'Q412', 'L291', 'R232'], #TP1
    ['ID00009', 'R822', 'W179', 'H017', 'P323', 'F298'], #TP2
    ['ID00007', 'Z011', 'X716', 'M948', 'W967', 'S702'], #TP3
    ['ID00004', 'N002', 'E396', 'N843', 'V935', 'S719'], #TP4
    ['ID10004', 'Q542', 'E423', 'N553', 'I453', 'S459'], #FN1
    ['NEW80187', 'J547', 'B222', 'G492', 'W673', 'S490'], #FP1
    ['NEW30110', 'N322', 'K685', 'T432', 'C225', 'W967'], #FP2
    ['NEW72832', 'F875', 'Q768', 'H822', 'Z154', 'X678'], 
    ['NEW30110', 'R560', 'C434', 'M687', 'Q689', 'Q863'],
    ['NEW81243', 'R762', 'N687', 'A109', 'K476', 'R637'],
    ['NEW52689', 'A089', 'V733', 'W158', 'A640', 'H331'],
    ['NEW67368', 'Z079', 'J617', 'G878', 'W111', 'Q500'],
    ['NEW72348', 'J547', 'B222', 'G492', 'R551', 'S490'],
    ['NEW34469', 'Y990', 'H898', 'W673', 'L967', 'M829'],
]

# Create DataFrames
columns = ['id', 'col1', 'col2', 'col3', 'col4', 'col5']
df1 = pd.DataFrame(data1, columns=columns)
df2 = pd.DataFrame(data2, columns=columns)

# Show
print(df1)
print(df2)

In [9]:
dataset = SyntheticMatcherDataset(size=20, true_positive_ratio=0.70)
df1, df2 = dataset.df1, dataset.df2
expected = dataset.expected

evaluator = DatasetEvaluator(df1, df2, expected, threshold=3)
evaluator.evaluate()
evaluator.printResults()

Expected: {'gt': 5, 'tp': 3, 'fp': 2, 'fn': 2}
Ground Truth Size: 3
True Positives: 3
False Positives: 2
False Negatives: 0
Precision: 0.6000
Recall: 1.0000
Elapsed Time: 0.01 seconds


In [8]:
df1, df2

(          0     1     2     3     4     5
 0   ID00000  H057  O528  S989  P371  M235
 1   ID00001  P533  O138  L763  I140  W743
 2   ID00002  C093  J326  F722  X963  A862
 3   ID00003  O524  Z363  R491  E934  P710
 4   ID00004  B954  Z304  Q352  J046  D670
 5   ID00005  V030  A960  R416  P067  F628
 6   ID00006  Y329  L186  Y513  J243  Y319
 7   ID00007  U184  X267  D366  V941  D990
 8   ID00008  O257  T263  P446  Y511  X354
 9   ID00009  T311  K188  T131  L387  Y580
 10  ID00010  F948  C769  N747  D475  H421
 11  ID00011  P442  H406  L610  V542  O809
 12  ID00012  E234  K245  V663  J979  O744
 13  ID00013  H400  M273  I938  P226  K032
 14  ID00014  W445  S097  O298  K351  Z131
 15  ID00015  O069  X547  M369  A533  N813
 16  ID00016  S004  L227  N811  R618  N403
 17  ID00017  A283  H501  N315  K167  B217
 18  ID00018  X057  K592  N553  Z796  W436
 19  ID00019  D263  K011  M491  L281  U462,
             0     1     2     3     4     5
 0     ID00008  O257  T263  P446  B409  X354
 1    